In [3]:
import bz2
import json
import numpy as np
import pandas as pd
import spacy
import pickle
from pathlib import Path

from IPython.display import display, HTML

from ressources import config

In [5]:
RAW_DATA_FOLDER = config.RAW_DATA_FOLDER
GENERATED_DATA_FOLDER = config.GENERATED_DATA_FOLDER

In [ ]:
def sanitarization(word_list):
    nlp = spacy.load("en_core_web_sm")
    word_list_san = []   
    for film in word_list:
        doc_film_name = nlp(film)
        tokens = [token.text for token in doc_film_name]
        result = " ".join(tokens)
        word_list_san.append(result)
    return word_list_san

## WIKIDATA Dataset
Exploring and filtering of Wikidata Dataset

In [13]:
# opening the parquet data
wiki = pd.read_parquet(RAW_DATA_FOLDER / 'WIKIDATA')
wiki = wiki[['id','aliases', 'label','date_of_birth','gender']]
# Wikidata is unique so there's no double
print(wiki['id'].is_unique)
wiki.shape


True


(9055981, 5)

In [7]:
# Function to get rid of list in a column

def try_join(cell):
    try:
        return ''.join(map(str, cell))
    except TypeError:
        return np.nan

In [9]:
# converting gender column (type: list / object) to a str
wiki['genderlabel'] = [try_join(cell) for cell in wiki['gender']]

# replacing gender wikidata id by male and female
wiki['genderlabel'].replace("Q6581097", 'male', inplace = True)
wiki['genderlabel'].replace("Q6581072", 'female', inplace = True)

for item, row in wiki.iterrow():
    if wiki['genderlabel'] not in ('male', 'female'):
        wiki['genderlabel'] = 'Other'

In [10]:
# We see that male and female are not the only two genders registred in Wikidata
wiki['genderlabel'].value_counts()

male                5418464
female              1684170
Q1052281                887
Q6581097Q6581072        664
Q48270                  307
                     ...   
Q6581097Q281833           1
Q6581072Q1052281          1
Q7130936                  1
Q48270Q2449503            1
Q6636                     1
Name: genderlabel, Length: 106, dtype: int64

In [ ]:
# converting date_of_birth column (type: list / object) to a str
wiki['date_of_birth'] = [try_join(cell) for cell in wiki['date_of_birth']]

# TODO: parse date and time to extract only date and calculate age of the speaker